In [1]:
import pandas as pd
pd.set_option('display.max_column', 999)
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score , StratifiedKFold

import pymorphy2
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

/home/kirill/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def validate(x , y):
    model = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6 , scale_pos_weight = 1 /(new_train.Label.mean()) - 1)
    cv = StratifiedKFold(4 ,shuffle=True, random_state=99)
    score = cross_val_score(model , x , y , scoring='roc_auc' , cv=cv)
    print (score.mean() , score.std() , '\n')

### Загружаем данные

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
all_data = pd.concat([train , test])

sample = pd.read_csv('data/sample_submission.csv')

### pymorphy2

In [4]:
%%time
morph = pymorphy2.MorphAnalyzer()

def name_score(word):
    for p in morph.parse(word):
        if 'Name' in p.tag:
            return p.score
    return 0

def surn_score(word):
    for p in morph.parse(word):
        if 'Surn' in p.tag:
            return p.score
    return 0


all_data['pymorphy'] = all_data['Word'].apply(lambda x: morph.tag(x)[0])

all_data['pymorphy_animacy'] = all_data['pymorphy'].apply(lambda x: x.animacy)
all_data['pymorphy_POS'] = all_data['pymorphy'].apply(lambda x: x.POS)
all_data['pymorphy_case'] = all_data['pymorphy'].apply(lambda x: x.case)
all_data['pymorphy_number'] = all_data['pymorphy'].apply(lambda x: x.number)
all_data['pymorphy_gender'] = all_data['pymorphy'].apply(lambda x: x.gender)

all_data.drop('pymorphy' , axis=1 , inplace=True)

columns_to_one_hot = ['pymorphy_animacy', 'pymorphy_POS', 'pymorphy_case','pymorphy_number', 'pymorphy_gender']

for col in columns_to_one_hot:
    all_data[col] = LabelEncoder().fit_transform(list(all_data[col].fillna('nan')))

CPU times: user 43.7 s, sys: 96 ms, total: 43.8 s
Wall time: 44.1 s


### cross-val-score

In [5]:
new_train = all_data[all_data['Label'].notnull()]
new_test = all_data[all_data['Label'].isnull()]
validate(new_train.drop(['Label' , 'Word'] , axis=1) , new_train['Label'])

0.775381646843 0.00364617275544 



### Если добавить основные функции строк , isupper , isupper , len , и другие фишки, легко можно получить 0.88

In [ ]:
model = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6 , scale_pos_weight = 1 /(new_train.Label.mean()) - 1)
model.fit(new_train.drop(['Label' , 'Word'] , axis=1) , new_train['Label'])
sample['Prediction'] = model.predict_proba(new_test.drop(['Word' , 'Label'] , axis=1))[:,0]
sample.to_csv('submit.csv' , index=False)